**Scenario: Smart Email Responder**

Build an AI that routes emails intelligently based on content.

**Flow:**

LLM analyzes email intent (question/complaint/feedback/spam)
LLM determines urgency (high/medium/low)

**Conditional routing:**

- Spam → Auto-delete
- Complaint + High urgency → Apologetic response (LLM)
- Question + Technical → Detailed answer (LLM with RAG)
- Feedback → Thank you note (LLM)
- Low urgency → Queue for later


In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Optional, Any, Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from utils import AWSLLM
from email_setup import EmailClient
import time

c:\Learning Journey\Langgraph_Agentic_AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# moddel for the classifier
class Classifier(BaseModel):
    intent: Literal['Question', 'Complaint', 'Feedback', 'Spam']
    urgency: Optional[Literal['High', 'Medium', 'Low']]

# model for question and detection    
class Technical_Detector(BaseModel):
    question: Optional[str] = Field(description="Extract the Question from the email.")
    is_technical: bool = Field(description="Classify the question is technical or not.")

class IncomingEmail(BaseModel):
    sender: str
    subject: str
    body: str

class SendingEmail(BaseModel):
    subject: str = Field(description="professional subject for rending email")
    body: str = Field(description="Clear, Structured and Professional email body")

In [3]:
# define state for the workflow
class SmartEmailState(TypedDict):
    email_text: str
    recipient_email: Optional[str]
    email_id: str
    intent: Optional[str]
    urgency: Optional[str]
    is_spam: bool
    is_technical: Optional[bool]
    question: Optional[str]
    email_response: Optional[dict]
    response: Optional[str]

In [4]:
llm = AWSLLM()
client = EmailClient()

In [5]:
# classify the email basis on intent and urgency
def classifier(state: SmartEmailState):
    email = state["email_text"]
    
    response = llm.chat(email, Classifier)
    state['intent']= response.intent
    state['urgency'] = response.urgency
    
    return state

# a coditional function node to detect spam or not
def is_spam(state: SmartEmailState):
    intent = state["intent"]

    if intent == "Spam":
        return {'is_spam': True}
    else:
        return {'is_spam': False}

# delete the email is the email is spam
def delete(state: SmartEmailState):
    email_id = state['email_id']

    client.delete_email(email_id)
    state['response'] = "It is a Spam Mail. Mail Deleted"
    return state

def router(state: SmartEmailState):
    return state

def check_status(state: SmartEmailState):
    intent = state["intent"]
    urgency = state["urgency"]

    if intent == "Question":
        return "question"

    if intent == "Complaint" and urgency == "High":
        return "apology_response"

    if intent == "Complaint" and urgency != "High":
        return "queue"

    if intent == "Feedback":
        return "feedback"

# node function for retrive question from email and detect technical or not
def technical_detection(state: SmartEmailState):
    email = state["email_text"]

    response = llm.chat(email, Technical_Detector)
    state["question"] = response.question
    state["is_technical"] = response.is_technical

    return state

# conditional function for next node selection rag or simple llm
def check(state: SmartEmailState):
    is_tech = state["is_technical"]

    if is_tech == True:
        return True
    else:
        return False

# RAG response node function
def rag_answer(state: SmartEmailState):
    from RAG import KBRag

    user_question = state['question']   
    kbrag = KBRag(pdf_path="Recruitment_Portal_Knowledge_Base.pdf")
    rag_response = kbrag(user_question)
    
    state["email_response"] = llm.chat(rag_response, SendingEmail)
    return state

# if the question is not technical
def llm_response(state: SmartEmailState):
    intent = state["intent"]
    urgency = state["urgency"]
    email = state["email_text"]

    prompt = f"""You are a professional customer support assistant.
                Email:
                {email}
                Intent: {intent}
                Urgency: {urgency}
                Write a polite, helpful, and complete response.
                Keep the tone friendly and professional.
                """
    state["email_response"] = llm.chat(prompt, SendingEmail)
    return state

# function for apology response
def apology_response(state: SmartEmailState):
    email = state["email_text"]
    prompt = f"""You are a customer support assistant handling a complaint.
                Email:
                {email}
                Write a sincere apology.
                Acknowledge the issue and reassure the user that it will be addressed.
                Keep it calm, empathetic, and professional.
                """
    state["email_response"] = llm.chat(prompt, SendingEmail)
    return state

# feedback response function
def feedback_response(state: SmartEmailState):
    email = state["email_text"]
    prompt = f"""You are responding to customer feedback.
                Email:
                {email}
                Thank the user for their feedback.
                If positive, express appreciation.
                If negative, acknowledge and assure improvement.
                Keep the response short and respectful.
                """
    state["email_response"] = llm.chat(prompt, SendingEmail)
    return state

# if the urgency is low the the email should be in queue
def queue(state: SmartEmailState):
    email = state["email_text"]
    
    prompt = f"""You are a support assistant.
                Email:
                {email}
                Inform the user that their request requires further review.
                Assure them it has been forwarded to the appropriate team.
                Do not provide a solution.
                Be polite and professional.
                """
    state["email_response"] = llm.chat(prompt, SendingEmail)
    return state

def send_email(state: SmartEmailState):
    to = state["recipient_email"]
    email_response = state['email_response']
    subject = email_response.subject
    body = email_response.body

    client.send_email(to, subject, body)
    state['response'] = "Email sent successfully"
    return state

In [6]:
# Define Graph
graph = StateGraph(SmartEmailState)

graph.add_node('classifier', classifier)
graph.add_node('is_spam', is_spam)
graph.add_node('delete', delete)
graph.add_node('router', router)
graph.add_node('technical_detection', technical_detection)
graph.add_node('rag_answer', rag_answer)
graph.add_node('llm_response', llm_response)
graph.add_node('apology_response', apology_response)
graph.add_node('feedback_response', feedback_response)
graph.add_node('queue', queue)
graph.add_node('send_email', send_email)


In [7]:
# Edges
graph.add_edge(START, 'classifier')
graph.add_edge('classifier', 'is_spam')

graph.add_conditional_edges(
    "is_spam",
    lambda state: state["is_spam"],
    {
        True: "delete",
        False: "router"
    }
)

# Conditional edges for different type
graph.add_conditional_edges(
    "router",
    check_status,
    {
        'question': 'technical_detection',
        'apology_response': 'apology_response',
        'feedback': 'feedback_response',
        'queue': 'queue'
    }
)

# conditional node 
graph.add_conditional_edges(
    'technical_detection',
    check,
    {
        True: 'rag_answer',
        False: 'llm_response'
    }
)
graph.add_edge('rag_answer', 'send_email')
graph.add_edge('llm_response', 'send_email')
graph.add_edge('apology_response', 'send_email')
graph.add_edge('feedback_response', 'send_email')
graph.add_edge('queue', 'send_email')
graph.add_edge('send_email', END)


In [8]:
workflow = graph.compile()


In [9]:
def run_agent(client, workflow):
    print("Agent started. Press Ctrl + C to stop.")

    try:
        while True:
            incoming = client.receive_email()

            if incoming is None:
                print("No new emails. Waiting...")
                time.sleep(10)
                continue

            # Build initial state for your workflow
            state = {
                "email_text": incoming['subject'] + incoming['body'],
                "recipient_email": incoming['sender'],
                "email_id": incoming['email_id']
            }

            # Run your LangGraph workflow
            result = workflow.invoke(state)

            print(result)
            print(result['response'])

            time.sleep(30)

    except KeyboardInterrupt:
        print("\nShutting down agent gracefully...")

In [10]:
run_agent(client, workflow)

Agent started. Press Ctrl + C to stop.
No new emails. Waiting...
No new emails. Waiting...
{'email_text': 'Need Help With Interview ReschedulingHello team,\r\nI have an interview scheduled for tomorrow, but due to a personal emergency\r\nI won’t be able to attend.\r\nCan you please tell me what the rescheduling policy is and how many times\r\nI’m allowed to reschedule?\r\nThanks,\r\nPriya\r\n', 'recipient_email': 'suubha090@gmail.com', 'email_id': b'7', 'intent': 'Question', 'urgency': 'Medium', 'is_spam': False, 'is_technical': False, 'question': 'Need Help With Interview ReschedulingHello team, I have an interview scheduled for tomorrow, but due to a personal emergency I won’t be able to attend. Can you please tell me what the rescheduling policy is and how many times I’m allowed to reschedule? Thanks, Priya', 'email_response': SendingEmail(subject='Re: Need Help With Interview Rescheduling', body="Dear Priya,\n\nThank you for reaching out. We understand that personal emergencies can